In [1]:
import preprocessing_util
import os
import numpy as np
import pandas as pd
import torch

In [ ]:
def preprocess(data_dir, dest_dir, max_dimensions=[400,400,400], resample_resolution=[2,2,2]):
    train_images_dir = os.path.join(data_dir, 'train_images/')
    train_label_coordinates_dir = os.path.join(data_dir, 'train_label_coordinates.csv')
    train_series_descriptions_dir = os.path.join(data_dir, 'train_series_descriptions.csv')
    train_labels_dir = os.path.join(data_dir, 'train.csv')
    
    studies = os.listdir(train_images_dir)
    
    for study in studies:
        
        
    
    return